# Visualize Ground Truth Masks

This notebook visualizes the generated masks for the TuSimple Lane Detection dataset. 
It checks if masks exist, generates them if missing, and overlays color-coded lane masks on the original images.

In [ ]:
import os
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Configuration
DATA_DIR = r'C:\Users\Alex\Documents\Clase\Italia\Segundo_ano\ADAS\Project\TU_Simple_folder\TUSimple'
PROCESSED_DATA_DIR = os.path.join(DATA_DIR, r'processed\instance')
# Standard TuSimple structure
TRAIN_SET_DIR = os.path.join(DATA_DIR, 'train_set')

print(f"Data Directory: {DATA_DIR}")
print(f"Processed Data Directory: {PROCESSED_DATA_DIR}")

In [ ]:
def process_tusimple_data(data_dir, output_dir, json_files):
    """
    Generates instance segmentation masks from TuSimple JSON labels.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Created output directory: {output_dir}")

    for json_file in json_files:
        json_path = os.path.join(data_dir, json_file)
        if not os.path.exists(json_path):
            print(f"Warning: JSON file not found: {json_path}")
            continue
            
        print(f"Processing {json_file}...")
        with open(json_path, 'r') as openfile:
            lines = openfile.readlines()

        count = 0
        for line in lines:
            info = json.loads(line)
            raw_file = info['raw_file']
            lanes = info['lanes']
            h_samples = info['h_samples']
            
            # Create mask assuming 1280x720 (TuSimple standard)
            mask = np.zeros((720, 1280), dtype=np.uint8)

            # Draw lanes with different class IDs
            for i, lane in enumerate(lanes):
                if all(x == -2 for x in lane):
                    continue
                    
                points = []
                for x, y in zip(lane, h_samples):
                    if x != -2:
                        points.append((x, y))
                
                if len(points) > 1:
                    class_id = i + 1
                    # Draw lane curve
                    cv2.polylines(mask, [np.array(points, dtype=np.int32)], isClosed=False, color=class_id, thickness=10) 
                    
            # Save mask
            mask_rel_path = raw_file.replace('.jpg', '.png')
            mask_save_path = os.path.join(output_dir, mask_rel_path)
            
            os.makedirs(os.path.dirname(mask_save_path), exist_ok=True)
            cv2.imwrite(mask_save_path, mask)
            count += 1
        print(f"Generated {count} masks from {json_file}")

In [ ]:
# Check if masks exist, if not, generate them
if not os.path.exists(PROCESSED_DATA_DIR) or len(os.listdir(PROCESSED_DATA_DIR)) == 0:
    print("Processed masks not found. Generating them now...")
    # Standard TuSimple label files
    json_files = ['label_data_0313.json', 'label_data_0531.json', 'label_data_0601.json']
    process_tusimple_data(TRAIN_SET_DIR, PROCESSED_DATA_DIR, json_files)
else:
    print("Processed masks folder exists. Proceeding to visualization.")

In [ ]:
def visualize_gt(image_path, mask_path, class_colors):
    """
    Visualizes the original image, the ground truth mask, and the overlay.
    """
    if not os.path.exists(image_path):
        print(f"Image not found: {image_path}")
        return
    if not os.path.exists(mask_path):
        print(f"Mask not found: {mask_path}")
        return

    # Load Image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Load Mask
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    # Create Color Mask
    # Class 0 is background (black)
    color_mask = np.zeros_like(image)
    
    unique_classes = np.unique(mask)
    print(f"Classes found in mask: {unique_classes}")

    for class_id in unique_classes:
        if class_id == 0:
            continue
        if class_id < len(class_colors):
            color_mask[mask == class_id] = class_colors[class_id]
        else:
            print(f"Warning: Class ID {class_id} out of range for colors.")

    # Overlay
    overlay = cv2.addWeighted(image, 0.7, color_mask, 0.3, 0)

    # Plot
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.title("Original Image")
    plt.imshow(image)
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.title("Ground Truth Mask (Colorized)")
    plt.imshow(color_mask)
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.title("Overlay")
    plt.imshow(overlay)
    plt.axis('off')

    plt.show()

In [ ]:
# Define Colors for visualization (Background + 5 lanes)
# Note: Mask generation uses class_id = i + 1 (1-based index for lanes)
COLORS = [
    [0, 0, 0],       # 0: Background
    [0, 255, 0],     # 1: Green
    [255, 0, 0],     # 2: Red
    [0, 0, 255],     # 3: Blue
    [255, 255, 0],   # 4: Yellow
    [255, 0, 255]    # 5: Magenta
]

# Select a JSON file to browse
json_file = 'label_data_0313.json' 
json_path = os.path.join(TRAIN_SET_DIR, json_file)

if os.path.exists(json_path):
    print(f"Loading {json_path}...")
    with open(json_path, 'r') as f:
        lines = f.readlines()
        
    # Visualize first 5 samples
    count = 0
    for line in lines:
        if count >= 5: break
        
        info = json.loads(line)
        raw_file = info['raw_file']
        
        # Construct paths
        img_path = os.path.join(TRAIN_SET_DIR, raw_file)
        
        # Mask path
        mask_rel_path = raw_file.replace('.jpg', '.png')
        mask_path = os.path.join(PROCESSED_DATA_DIR, mask_rel_path)
        
        if os.path.exists(img_path) and os.path.exists(mask_path):
            print(f"\nSample {count+1}:")
            print(f"Image: {img_path}")
            print(f"Mask:  {mask_path}")
            visualize_gt(img_path, mask_path, COLORS)
            count += 1
        else:
            # Skip files if not found (e.g. if generation failed or partial)
            continue
else:
    print(f"JSON file not found: {json_path}. Please check DATA_DIR path.")